In [1]:
!nvidia-smi

Sat Dec  3 03:58:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# ! pip install textattack
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 52.0 MB/s 
     |████████████████████████████████| 7.6 MB 50.1 MB/s 


In [3]:
import textattack
import transformers

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)
train_dataset = textattack.datasets.HuggingFaceDataset("imdb", split="train")
eval_dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 29.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp4b1e7ea1.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset imdb, split train.


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset imdb, split test.


In [6]:
print(len(train_dataset))
print(len(eval_dataset))

25000
25000


In [8]:
training_args = textattack.TrainingArgs(
    num_epochs=3,
    num_clean_epochs=1,
    num_train_adv_examples=-1,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    log_to_tb=True,
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    train_dataset,
    eval_dataset,
    training_args
)
trainer.train()

textattack: Writing logs to ./outputs/2022-12-03-02-18-34-616223/train_log.txt.
textattack: Wrote original training args to ./outputs/2022-12-03-02-18-34-616223/training_args.json.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 25000
textattack:   Num epochs = 3
textattack:   Num clean epochs = 1
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 64
textattack:   Gradient accumulation steps = 8
textattack:   Total optimization steps = 1955
textattack: ==========================================================
textattack: Epoch 1
textattack: Running clean epoch 1/1
Loss 0.37717: 

KeyboardInterrupt: ignored

# num_epochs=3,
    num_clean_epochs=1,
    num_train_adv_examples=10,

textattack: Writing logs to ./outputs/2022-12-01-12-22-42-386079/train_log.txt.
textattack: Wrote original training args to ./outputs/2022-12-01-12-22-42-386079/training_args.json.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 25000
textattack:   Num epochs = 3
textattack:   Num clean epochs = 1
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 64
textattack:   Gradient accumulation steps = 8
textattack:   Total optimization steps = 1173
textattack: ==========================================================
textattack: Epoch 1
textattack: Running clean epoch 1/1
Loss 0.38819: 100%|██████████| 3125/3125 [37:12<00:00,  1.40it/s]
textattack: Train accuracy: 80.33%
textattack: Eval accuracy: 92.25%
textattack: Best score found. Saved model to ./outputs/2022-12-01-12-22-42-386079/best_model/
textattack: ==========================================================
textattack: Epoch 2
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 10 / 0 / 1 / 11: 100%|██████████| 10/10 [03:11<00:00, 19.14s/it]
textattack: Total number of attack results: 11
textattack: Attack success rate: 100.00% [10 / 10]


Loss 0.28172: 100%|██████████| 3127/3127 [37:11<00:00,  1.40it/s]
textattack: Train accuracy: 93.53%
textattack: Eval accuracy: 93.77%
textattack: Best score found. Saved model to ./outputs/2022-12-01-12-22-42-386079/best_model/
textattack: ==========================================================
textattack: Epoch 3
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10: 100%|██████████| 10/10 [02:11<00:00, 13.16s/it]
textattack: Total number of attack results: 10
textattack: Attack success rate: 100.00% [10 / 10]


Loss 0.20509: 100%|██████████| 3127/3127 [37:11<00:00,  1.40it/s]
textattack: Train accuracy: 98.68%
textattack: Eval accuracy: 93.83%
textattack: Best score found. Saved model to ./outputs/2022-12-01-12-22-42-386079/best_model/
textattack: Wrote README to ./outputs/2022-12-01-12-22-42-386079/README.md.





# num_epochs=3,
    num_clean_epochs=1,
    num_train_adv_examples=50,
textattack: Writing logs to ./outputs/2022-12-02-09-32-46-391582/train_log.txt.
textattack: Wrote original training args to ./outputs/2022-12-02-09-32-46-391582/training_args.json.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 25000
textattack:   Num epochs = 3
textattack:   Num clean epochs = 1
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 64
textattack:   Gradient accumulation steps = 8
textattack:   Total optimization steps = 1175
textattack: ==========================================================
textattack: Epoch 1
textattack: Running clean epoch 1/1
Loss 0.39639: 100%|██████████| 3125/3125 [39:22<00:00,  1.32it/s]
textattack: Train accuracy: 80.42%
textattack: Eval accuracy: 92.34%
textattack: Best score found. Saved model to ./outputs/2022-12-02-09-32-46-391582/best_model/
textattack: ==========================================================
textattack: Epoch 2
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 50 / 0 / 3 / 53: 100%|██████████| 50/50 [13:33<00:00, 16.27s/it]
textattack: Total number of attack results: 53
textattack: Attack success rate: 100.00% [50 / 50]


Loss 0.28779: 100%|██████████| 3132/3132 [39:42<00:00,  1.31it/s]
textattack: Train accuracy: 93.39%
textattack: Eval accuracy: 93.70%
textattack: Best score found. Saved model to ./outputs/2022-12-02-09-32-46-391582/best_model/
textattack: ==========================================================
textattack: Epoch 3
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 50 / 0 / 1 / 51: 100%|██████████| 50/50 [18:39<00:00, 22.39s/it]
textattack: Total number of attack results: 51
textattack: Attack success rate: 100.00% [50 / 50]


Loss 0.20882: 100%|██████████| 3132/3132 [39:42<00:00,  1.31it/s]
textattack: Train accuracy: 98.61%
textattack: Eval accuracy: 93.72%
textattack: Best score found. Saved model to ./outputs/2022-12-02-09-32-46-391582/best_model/
textattack: Wrote README to ./outputs/2022-12-02-09-32-46-391582/README.md.

In [3]:
! pip install -q datasets

     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 71.8 MB/s 
     |████████████████████████████████| 132 kB 73.9 MB/s 
     |████████████████████████████████| 127 kB 78.3 MB/s 


~~~~~~~~~~~~~~~~~~~
# ----------------------------------
~~~~~~~~~~~~~~~~~~~~~~~

In [4]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, Value, ClassLabel, Features
from transformers import DataCollatorWithPadding, AutoTokenizer, pipeline

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [7]:
data = load_dataset("imdb")
df_train = pd.DataFrame.from_dict(data['train']).sample(25000)
df_test = pd.DataFrame.from_dict(data['test']).sample(25000)
features = Features({
    "text": Value("string"), 
    "label": ClassLabel(num_classes=2, names=['neg','pos']),
    "__index_level_0__": Value("string") 
    })

# recreate the data object using the smaller df's
data = DatasetDict({
    'train': Dataset.from_pandas(df_train, features=features),
    'test': Dataset.from_pandas(df_test, features=features),
    })

# remove index col (seems to be coming in from pandas for some reason)
data = data.remove_columns(["__index_level_0__"])

  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# tokenize the data
tokenized_data = data.map(preprocess_function, batched=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [11]:
tokenized_data["train"]


Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [12]:
tokenized_data["test"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [13]:
from datasets import load_metric
import numpy as np
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-13-165db97bb7a9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [14]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
batch_size = 8
learning_rate = 2e-5
num_epochs = 3
weight_decay = 0.01
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    
training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        fp16=True,
        evaluation_strategy="epoch",
        weight_decay=weight_decay,
        save_steps=5000,
    )
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )   

trainer.train()

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy
1,0.253400,0.230858,0.929360
2,0.156600,0.281904,0.941120
3,0.070800,0.314462,0.941560


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-5000
Configuration saved in ./results/checkpoint-5000/config.json
Model weights saved in ./results/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-5000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-5000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8
The

TrainOutput(global_step=9375, training_loss=0.17428064595540366, metrics={'train_runtime': 4091.9444, 'train_samples_per_second': 18.329, 'train_steps_per_second': 2.291, 'total_flos': 1.85844177287784e+16, 'train_loss': 0.17428064595540366, 'epoch': 3.0})

Epoch	Training Loss	Validation Loss	Accuracy\
1	0.253400	0.230858	0.929360\
2	0.156600	0.281904	0.941120\
3	0.070800	0.314462	0.941560\
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-5000
Configuration saved in ./results/checkpoint-5000/config.json
Model weights saved in ./results/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-5000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-5000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


TrainOutput(global_step=9375, training_loss=0.17428064595540366, metrics={'train_runtime': 4091.9444, 'train_samples_per_second': 18.329, 'train_steps_per_second': 2.291, 'total_flos': 1.85844177287784e+16, 'train_loss': 0.17428064595540366, 'epoch': 3.0})

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ---------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~